In [1]:
#attributions, explanations, etc. are in the College search IL notebook
#get all the data from the CollegeScorecard API
import requests
import math
import numpy as np
from IPython.display import display
import pandas as pd

key = "nyiNevdtUIEMrkovbB6bYPDsdi8V4rRBSEzXtN9s"
url_base = "https://api.data.gov/ed/collegescorecard/v1/schools/"
fields = ','.join([
    "location.lat","location.lon",
    "school.religious_affiliation","school.name","school.alias","school.city","school.state","school.zip","school.ownership_peps","school.school_url","school.locale","school.institutional_characteristics.level",
    "latest.admissions.admission_rate.overall","latest.admissions.sat_scores.midpoint.critical_reading","latest.admissions.sat_scores.midpoint.writing","latest.admissions.sat_scores.midpoint.math","latest.admissions.act_scores.midpoint.cumulative",
    "latest.academics.program_percentage.computer","latest.academics.program_percentage.mathematics",
    "latest.student.demographics.median_hh_income","latest.student.size","latest.student.demographics.female_share","latest.student.demographics.race_ethnicity.white","latest.student.demographics.race_ethnicity.black","latest.student.demographics.race_ethnicity.hispanic","latest.student.demographics.race_ethnicity.asian",
    "latest.cost.net_price.public.by_income_level.48001-75000","latest.cost.net_price.public.by_income_level.75001-110000","latest.cost.attendance.academic_year","latest.cost.tuition.in_state","latest.cost.tuition.out_of_state",
    "latest.aid.federal_loan_rate",
    "latest.earnings.10_yrs_after_entry.median","latest.earnings.6_yrs_after_entry.median",
])

def get_col_data(page):
    p = {
    "school.men_only":"0", "school.women_only":"0", "school.online_only":"0", "school.operating":"1",
    "fields":fields,
    "page":page, 
    "api_key": key
    }
    resp = requests.get(url=url_base, params=p)
    return resp.json()

try:
    metadata = get_col_data(0)['metadata']
except:
    print(get_col_data(0)) #print the error

all_pages = []
for i in range(math.ceil(metadata['total']/metadata['per_page'])):
    all_pages.extend(get_col_data(i)['results']) #add data from all pages

In [2]:
#Turn it into dataframe, make it look nice
collegeInfo = pd.DataFrame(all_pages).set_index('school.name').fillna(value=np.nan)
collegeInfo = collegeInfo[~collegeInfo.index.duplicated(keep='last')] #remove duplicates

#If there is a year (latest in this case), remove it from column name. Also remove the dev category
collegeInfo = collegeInfo.rename(columns = lambda x: '.'.join(x.split('.')[2:]) if 'latest' in x else x) #turn latest.admissions.admission_rate.overall into admission_rate.overall.
collegeInfo = collegeInfo.reindex(sorted(collegeInfo.columns), axis=1) #sort column names alphabetically to make it easier to rename them

#manually rename the columns
#NOTE: median is used for all the calculations aside from net price
newNames = [
    'Salary 10 years after entry','Salary 6 years after entry',
    'Cumulative ACT score','Admission rate','Overall annual cost of attendance',
    '% female','Household income','% Asian','% Black','% Hispanic','% White','% with a federal loan',
    'Latitude','Longtitude',
    'Net price for $48001-75000 household income','Net price for $75001-110000 household income',
    '% students in computer science','% students in math',
    'SAT Reading','SAT Math','SAT Writing',
    'Aliases',
    'City','Level of institution','Locale',
    'Ownership','Religious affiliation','Website',
    'State','ZIP',
    '# students',
    'In-state tuition','Out-of-state tuition'
]
collegeInfo.columns = newNames

#manually rename the index
collegeInfo.index.names = ['Name'] #instead of school.name

#reoder the column names with a custom order
newOrder = [
    'Religious affiliation',
    'Salary 10 years after entry','Salary 6 years after entry',
    'Admission rate',
    '% female','% Asian','% Black','% Hispanic','% White',
    '% students in computer science','% students in math',
    'Cumulative ACT score','SAT Reading','SAT Math','SAT Writing',
    'Aliases',
    'City',
    'Level of institution',
    'Locale',
    'Ownership',
    'Website',
    'State',
    'ZIP','Latitude','Longtitude',
    '# students',
    'Household income',
    '% with a federal loan',
    'Net price for $48001-75000 household income','Net price for $75001-110000 household income',
    'Overall annual cost of attendance','In-state tuition','Out-of-state tuition'
]
collegeInfo = collegeInfo[newOrder]

#Ownership, level of instituion, and locale columns contain numbers that correspond to certain values - map them
#Religion will be mapped later
localeDic = {
    11:'Large City',12:'Midsize City',13:'Small City',
    21:'Large Suburb',22:'Midsize Suburb',23:'Small Suburb',
    31:'Fringe Town',32:'Distant Town',33:'Remote Town',
    41:'Fringe Rural',42:'Distant Rural',43:'Remote Rural',
}
ownDic = {1:'Public',2:'Private, Nonprofit',3:'Proprietary'}
levelDic = {1:'4-year',2:'2-year',3:'Less-than-2-year'}

collegeInfo['Locale'] = collegeInfo['Locale'].map(localeDic)
collegeInfo['Ownership'] = collegeInfo['Ownership'].map(ownDic)
collegeInfo['Level of institution'] = collegeInfo['Level of institution'].map(levelDic)

states = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}
collegeInfo['State'] = collegeInfo['State'].map(states) #rename state abbreviations to full names  

In [3]:
#combine it with another dataset that i got straight from the IPEDS. Data is from 2018/19
ipedsCols = [
    'instnm','Address',
    "Number of students receiving a Doctor's degree (DRVC2018)",
    "Number of students receiving a Master's degree (DRVC2018)",
    "Number of students receiving a Bachelor's degree (DRVC2018)",
    "Number of students receiving an Associate's degree (DRVC2018)",
    'Graduation rate','Average amount of aid awarded'
]
ipeds = pd.read_csv(r'C:\Users\timkh\OneDrive\Desktop\Programming\Projects\Colleges\IPEDS.csv',usecols=ipedsCols).set_index('instnm').replace(r'^\s*$', np.nan, regex=True) # replace field that's entirely space (or empty) with NaN
#set the graduate # to the sum of # of PhD, # of Masters, # of Bachleors, and # of Associates graduating. This ensures the percentages are correct
ipeds['Graduate #'] = ipeds["Number of students receiving a Doctor's degree (DRVC2018)"]+ipeds["Number of students receiving a Master's degree (DRVC2018)"]+ipeds["Number of students receiving a Bachelor's degree (DRVC2018)"]+ipeds["Number of students receiving an Associate's degree (DRVC2018)"]
ipeds["% receiving Doctor's"] = ipeds["Number of students receiving a Doctor's degree (DRVC2018)"]/ipeds['Graduate #']
ipeds["% receiving Master's"] = ipeds["Number of students receiving a Master's degree (DRVC2018)"]/ipeds['Graduate #']
ipeds["% receiving Bachelor's"] = ipeds["Number of students receiving a Bachelor's degree (DRVC2018)"]/ipeds['Graduate #']
ipeds["% receiving Associate's"] = ipeds["Number of students receiving an Associate's degree (DRVC2018)"]/ipeds['Graduate #']
#drop the columns that aren't needed anymore
ipeds.drop(["Number of students receiving a Doctor's degree (DRVC2018)",
    "Number of students receiving a Master's degree (DRVC2018)",
    "Number of students receiving a Bachelor's degree (DRVC2018)",
    "Number of students receiving an Associate's degree (DRVC2018)", "Graduate #"],axis=1,inplace=True)

#merge
collegeInfo = pd.merge(collegeInfo,ipeds,how='left',left_index=True,right_index=True)

In [4]:
#multiply all percentages by 100 and round numbers
for col in collegeInfo.columns:
    if '%' in col or col == 'Admission rate':
        collegeInfo[col] = collegeInfo[col].map(lambda x: x*100 if x<=1 else x)
collegeInfo = collegeInfo.round(2)

#add a new column with the distance of the college from home
def get_distance(row):
    lat1 = math.radians(row['Latitude'])
    lon1 = math.radians(row['Longtitude'])
    lat2=math.radians(42.04)
    lon2=math.radians(-87.94)
    
    r = 3958.8 #radius of the Earth
    dlon = lon2 - lon1 #change in coordinates
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2 #Haversine formula
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = r * c
    return round(distance,2)
collegeInfo['Approximate distance from home (mi)'] = collegeInfo.apply(get_distance,axis=1)
collegeInfo.drop(['Latitude','Longtitude'],axis=1,inplace=True) #delete the columns that aren't needed anymore
collegeInfo = collegeInfo[collegeInfo['Religious affiliation'].apply(math.isnan)].dropna(thresh=20) #remove religious universities
collegeInfo.drop('Religious affiliation',axis=1,inplace=True)

In [27]:
pd.options.mode.chained_assignment = None #avoid the false warnings
toNorm = collegeInfo.select_dtypes('number')
toNorm['% Asian + % White normalized'] = toNorm['% White'] + toNorm['% Asian']
#1 - calculating deviation
toNorm['Admission rate deviation'] = toNorm['Admission rate'].map(lambda x: abs(x-65))
toNorm['% female deviation'] = toNorm['% female'].map(lambda x: abs(x-50))
toNorm['% Black deviation'] = toNorm['% Black'].map(lambda x: abs(x-5))
toNorm['% Asian + % White deviation'] = toNorm['% Asian + % White normalized'].map(lambda x: abs(x-75))
toNorm['% students in computer science deviation'] = toNorm['% students in computer science'].map(lambda x: abs(x-20))
toNorm['% students in math deviation'] = toNorm['% students in math'].map(lambda x: abs(x-10))

#2 - normalizing, calculating score, multipying by weights
normalized=((toNorm-toNorm.min())/(toNorm.max()-toNorm.min()))*100 #Found this method by looking up 'pandas standardize each column', 2nd result
normalized['Score'] = (
            normalized['Salary 10 years after entry'].fillna(0)+ #extremely important - weight is 1
            normalized['Salary 6 years after entry'].fillna(0)*0.6+
            normalized['Cumulative ACT score'].fillna(0)*0.8+
            normalized['SAT Reading'].fillna(0)*0.6+
            normalized['SAT Math'].fillna(0)*0.9+
            normalized['# students'].fillna(0)*0.4+ #many universities have NaN SAT writing, so I removed it
            normalized['Graduation rate'].fillna(0)*0.7-
            
            normalized['Overall annual cost of attendance'].fillna(100)*0.9-
            #normalized['In-state tuition'].fillna(100)*0.8-
            normalized['Admission rate deviation'].fillna(100)*0.5-
            normalized['% Black deviation'].fillna(100)*0.7-
            normalized['% Asian + % White deviation'].fillna(100)*0.7-
            normalized['% female deviation'].fillna(100)*0.4-
            normalized['% students in computer science deviation'].fillna(100)- #extremely important - weght is 1
            normalized['% students in math deviation'].fillna(100)*0.5
)


#add the word 'normalized' to the columns with 'deviation' in their names. 
#The other columns (aside from Score) will automatically have the word 'normalized' added to them as a suffix while merging, since the orginal collegeInfo df has the same column names. 
normalized.rename(lambda x: x + ' normalized' if 'deviation' in x else x,axis = 1,inplace=True)
normalizedExtended = pd.merge(normalized,collegeInfo,how='left',left_index=True,right_index=True,suffixes=(' normalized','')) 
#could really use any method (not just left) since the goal of this merge is  just to add new columns
def subtractTuition(row):
    if row['State'] == 'Illinois':
        row['Score'] -= row['In-state tuition normalized']*0.8 if not math.isnan(row['In-state tuition normalized']) else 80 #subtract 80 if it's NaN
    else:
        row['Score'] -= row['Out-of-state tuition normalized']*0.8 if not math.isnan(row['Out-of-state tuition normalized']) else 80
    return row['Score']

normalizedExtended['Score'] = normalizedExtended.apply(subtractTuition,axis=1) #also subtract the tution, otherwise the
#system will favor the 'good but expensive' universities like Harvard and MIT, whereas I'm looking for a balance between price and quality
with pd.option_context('display.max_columns', None):  #show all columns
    display(normalizedExtended.nlargest(20,columns=['Score']))
    #display(normalizedExtended.sort_values(by='Score',ascending=False).index.get_loc('Northwestern University'))
    #UIUC - 2nd
    #U of C - 43rd
    #Northwestern - 67th
    #IIT - 146th
    #UIC - 171st

,Religious affiliation normalized,Salary 10 years after entry normalized,Salary 6 years after entry normalized,Admission rate normalized,% female normalized,% Asian normalized,% Black normalized,% Hispanic normalized,% White normalized,% students in computer science normalized,% students in math normalized,Cumulative ACT score normalized,SAT Reading normalized,SAT Math normalized,SAT Writing normalized,# students normalized,Household income normalized,% with a federal loan normalized,Net price for $48001-75000 household income normalized,Net price for $75001-110000 household income normalized,Overall annual cost of attendance normalized,In-state tuition normalized,Out-of-state tuition normalized,Graduation rate normalized,Average amount of aid awarded normalized,% receiving Doctor's normalized,% receiving Master's normalized,% receiving Bachelor's normalized,% receiving Associate's normalized,Approximate distance from home (mi) normalized,% Asian + % White normalized,Admission rate deviation normalized,% female deviation normalized,% Black deviation normalized,% Asian + % White deviation normalized,% students in computer science deviation normalized,% students in math deviation normalized,Score,Religious affiliation,Salary 10 years after entry,Salary 6 years after entry,Admission rate,% female,% Asian,% Black,% Hispanic,% White,% students in computer science,% students in math,Cumulative ACT score,SAT Reading,SAT Math,SAT Writing,Aliases,City,Level of institution,Locale,Ownership,Website,State,ZIP,# students,Household income,% with a federal loan,Net price for $48001-75000 household income,Net price for $75001-110000 household income,Overall annual cost of attendance,In-state tuition,Out-of-state tuition,Address,Graduation rate,Average amount of aid awarded,% receiving Doctor's,% receiving Master's,% receiving Bachelor's,% receiving Associate's,Approximate distance from home (mi)
Georgia Institute of Technology-Main Campus,NaN,58.326217,50.629496,21.53,37.250578,21.701827,6.97,7.24,48.04,16.98,9.115770,88.888889,88.000000,89.411765,80.722892,18.960958,73.040668,26.53,49.724897,57.976499,26.531363,16.133128,43.952778,87.0,23.029503,9.532804,43.02,49.41,0.00,7.404742,69.54,66.876923,25.686151,2.073684,7.280000,3.7750,89.989990,203.499052,NaN,79100.0,65500.0,21.53,37.74,21.50,6.97,7.24,48.04,16.98,1.00,33.0,715.0,750.0,685.0,Georgia Tech,Atlanta,4-year,Large City,Public,www.gatech.edu,Georgia,30332-0530,15201.0,77933.0,26.53,15988.0,17819.0,28501.0,12424.0,33020.0,225 North Ave,87.0,12716.0,7.57,43.02,49.41,0.00,603.14
University of Illinois at Urbana-Champaign,NaN,43.296328,34.082734,62.16,49.579920,19.127889,6.04,12.03,44.22,4.67,34.275296,74.074074,69.333333,74.117647,69.879518,41.130099,72.270804,34.90,52.888583,75.041967,28.242680,19.739579,42.121188,84.0,26.068432,10.842463,29.70,61.69,0.00,1.623742,63.17,4.369231,1.089462,1.094737,15.773333,19.1625,62.362362,200.581344,NaN,61500.0,47100.0,62.16,49.48,18.95,6.04,12.03,44.22,4.67,3.76,29.0,645.0,685.0,640.0,Illinois|Illinios|Ilinois|Ilinios|Urbana|Champ...,Champaign,4-year,Small City,Public,www.illinois.edu/,Illinois,61820-5711,32974.0,77278.0,34.90,16954.0,23817.0,30082.0,15094.0,31664.0,601 E John Street,84.0,14328.0,8.61,29.70,61.69,0.00,134.89
Massachusetts Institute of Technology,NaN,80.187874,65.647482,6.74,36.672968,28.131624,6.15,15.23,31.67,32.63,70.647220,96.296296,96.000000,98.823529,92.771084,5.675440,72.444758,10.56,NaN,NaN,68.669156,69.362725,69.362725,94.0,90.411914,23.913865,50.25,30.76,0.00,10.592471,59.54,89.630769,26.838466,1.210526,20.613333,15.7875,22.422422,196.106147,NaN,104700.0,82200.0,6.74,37.19,27.87,6.15,15.23,31.67,32.63,7.75,35.0,745.0,790.0,735.0,MIT M.I.T.,Cambridge,4-year,Midsize City,"Private, Nonprofit",web.mit.edu/,Massachusetts,02139-4307,4550.0,77426.0,10.56,NaN,NaN,67430.0,51832.0,51832.0,77 Massachusetts Avenue,94.0,48459.0,18.99,50.25,30.76,0.00,861.34
Harvard University,NaN,67.378309,54.946043,4.73,50.283554,19.400424,8.03,11.33,40.5